## Algebraic model

::: {.callout-note collapse="true"}
## Model Sketch

![](sci-group-sketch.jpg)

:::

The key ingredients of the model are our resaerch groups $G$ with a number of non-programmers $n$ and programmers $p$. In a data-driven world, we assume that learning to code confer a large benfit programmers over non-programmer such that $\alpha << \beta$. There is a constant rate of influx of students who do not know how to learn to code in research group $\mu$. There is a cost of learning to code $c(p,n)$ which depends on the number of programmers and non-programmers in the group. 

Our master equation:

\begin{align*}
\frac{d}{dt}G_{n,p} &= \mu(G_{n-1,p} - G_{n,p}) + \nu_n \Big((n+1)G_{n+1,p}-nG_{n,p}\Big) \\
                           &+ \Big[ \tau(n+1,p-1)(1-c(p-1,n+1)G_{n+1,p-1} - \tau(n,p)G_{n,p} \Big] \\
                   &+ \nu_p\Big((p+1)G_{n,p+1} - pG_{n,p} \Big)
\end{align*}


## Cost functions



```{ojs}
//| panel: sidebar

// i == p == # programmers
function cost_prog(n, i, c_0) { return c_0 * Math.exp(-i/n); }

max_gr_size = 20
viewof N = Inputs.range([0, max_gr_size], {value: 20, step: 1, label: "group Size"})
viewof coder = Inputs.range([1, max_gr_size], {value: 10, step: 1, label: "# coder"})
viewof c_0 = Inputs.range([0, 5], {value: 1, step: 0.1, label: "c₀"})
viewof nc = Inputs.range([1, max_gr_size], {value: (N-coder), step: 1., label: "# non-coder", disabled:true})
```


::: {.panel-tabset}

## c(n,p) = c₀ * exp(-p/n)


```{ojs}
non_coder = N - coder
xs = [...Array(N).keys()];
ys = xs.map(x => cost_prog(non_coder, x, c_0))


Plot.lineY(ys).plot({
    height: 400, width: 450, grid: true, 
    y: { label: "↑ c(n,p)", domain:[0,1] }, 
    x: { label: "p/n →" }
})
```


$p/n$ $\Rightarrow$ ${coder}`/` ${non_coder} = ${(coder/(non_coder)).toFixed(2)} <br><br>
$c(n,p)$ = ${(cost_prog2(coder, N-coder)).toFixed(2)} (c=1 means that non-coders always fail to learn to code; c=0 means non-coders always succeed)<br><br>

::: 

## Group benefits



```{ojs}
//| panel: sidebar

// i == p == # programmers
function tau(n, i, alpha, beta) { 
    const c = cost_prog(n, i, 1)
    return Math.exp(-alpha + beta*(1 - c))
}

tau_max_gr_size = 20
viewof alpha = Inputs.range([2, 4], {value: 1., step: 1, label: "α", format: x => 10**-x})
viewof beta = Inputs.range([1, 3], {value: 1., step: 1, label: "β", format: x => 10**-x})
viewof tau_N = Inputs.range([0, tau_max_gr_size], {value: 20, step: 1, label: "group Size"})
viewof tau_coder = Inputs.range([1, tau_max_gr_size], {value: 10, step: 1, label: "# coder"})
viewof tau_nc = Inputs.range([1, max_gr_size], {value: (tau_N-tau_coder), step: 1., label: "# non-coder", disabled:true})
```


::: {.panel-tabset}

## τ(n,p) 1


```{ojs}
tau_non_coder = tau_N - tau_coder
tau_xs = [...Array(tau_N).keys()];
tau_ys = tau_xs.map(x => tau(tau_non_coder, x, 10**-alpha, 10**-beta))

Plot.lineY(tau_ys).plot({
    height: 400, width: 450, grid: true, 
    y: { label: "↑ τ(α,β;n,p)" }, 
    x: { label: "p/n →" }
})
```


::: 


::: {.callout-tip collapse="true"}
## Initialization scheme


In [ ]:
using OrdinaryDiffEq, Plots, Distributions, RecursiveArrayTools

function initialize_u0(;N::Int=20, M::Int=20, p::Float64=0.01)
  
  function make_draw()
    G = zeros(N, N+1)
    for _ in 1:N
      n = rand(1:N)
      i = sum(collect(rand(Binomial(1, p), N))[1]) # how many total programmers?
      G[n, i+1] += 1 # everytime combination G[ℓ,i], count +1
    end
    return G
  end

  G = make_draw()
  
  # making sure we start with zero programmers but we have varying group size
  while sum(G[:,2:N+1]) != 0
    G = make_draw()
  end

  G = G ./ N # normalized by tot number of groups
  
  hm = heatmap(G, xlabel="# non-coders", ylabel="# coders", title="Frac groups with p coders and n non-coder", c = cgrad([:orange, :blue], [0.1, 0.3, 0.8]))

  # ArrayPartition are nice because we can access the level such as G.x[ℓ][i]
  return ArrayPartition(Tuple([G[n,:] for n=1:N])), hm
end

μ  = 0.001   # inflow new students-non coders
νₙ = 0.01   # death rate non-coders
νₚ = 0.05   # death rate coders
α  = 0.01    # benefits non coders
β  = 0.1     # benefits coders
p  = [μ, νₙ, νₚ, α, β]

n,M = 20, 1000
u₀, hm = initialize_u0(N=n, M=M, p=0.01) # initialisation - fraction de groupe
tspan = (0., 100.)

hm

:::

## Julia model


In [ ]:
c(n, i) = 1 * exp(-i / n) # cost function
τ(n, i, α, β) = exp(-α + β*(1 - c(n, i))) # group benefits

function life_cycle_research_groups!(du, u, p, t)
  G, N, P = u, length(u.x), length(first(u.x))
  μ, νₙ, νₚ, α, β = p
  
  for n=1:N, i=1:P
    non_coder, coder = n-1, i-1 # we distinguish indices from actual values, e.g. 
    
    if (n < non_coder)

      n > 1 && (du.x[n][i]  = μ*(G.x[n-1][i] - G.x[n][i]) + νₙ*((non_coder+1) * G.x[n+1][i] - non_coder * G.x[n][i]) )
      
      i > 1 && ( du.x[n][i] += τ(non_coder+1, coder-1, α, β)*(1 - c(non_coder+1, coder-1))*G.x[n+1][i-1] - τ(non_coder, coder, α, β)*G.x[n][i]*G.x[n][i] )
      
      i < coder && ( du.x[n][i] += νₚ*( (coder+1)*G.x[n][i+1] - coder*G[n][i] ) )

      du.x[n][i] += τ(n+1, i, α, β)*(1 - c(non_coder, coder))*G.x[n+1][i]
    
    end
  end
end

In [ ]:
prob = ODEProblem(life_cycle_research_groups!, u₀, tspan, p)
sol = solve(prob, Tsit5(), saveat = 1., reltol=1e-8, abstol=1e-8)

In [ ]:
N = length(sol[1].x)
i = length(sol[1].x[1])
I = zeros(N, length(sol.t))
    
for t in 1:length(sol.t)
  for n in 1:N
    G_nil = sol[t].x[n]
    I[n, t] = sum((collect(0:(i-1)) / n) .* G_nil) / sum(G_nil)
  end
end

In [ ]:
sol[10]

In [ ]:
new_run = []
for n in 1:N
  tmp_dat = [Dict("N" => n, "value" => I[n, i], "timesteps" => i) for i in 1:3000]
  x = round.([i["value"] for i in tmp_dat], digits=5) # To lighten the output file we only keep unique `y` values. 
  idx = unique(z -> x[z], 1:length(x))
  push!(new_run, tmp_dat[idx] )
end

if isfile("data.json")
  get!(old_run, join(p, "_"), vcat(new_run...))
  new_run = old_run
else
  new_run = Dict(join(p, "_") => vcat(new_run...))
end
      
open("data.json", "w") do f 
  write(f, JSON.json(new_run))
end

## Output


```{ojs}
data = FileAttachment("data.json").json()
p = Object.keys(data).map(d => d.split("_")) //unpack params
```

```{ojs}
//| panel: sidebar

minmax = (p, i) => d3.extent(p.map(d => parseFloat(d[i])))

viewof mu    = Inputs.range(minmax(p,0), {step: 0.03, label: "μ", value:"0.0001"})
viewof nu_n  = Inputs.range(minmax(p,0), {step: 0.05, label: "β", value:"0.07"})
viewof nu_p  = Inputs.range(minmax(p,1), {step: 0.1,  label: "γ", value:"1.0"})
viewof alpha = Inputs.range(minmax(p,2), {step: 0.15, label: "ρ", value:"0.1"})
viewof beta  = Inputs.range(minmax(p,3), {step: 0.05, label: "b", value:"0.18"})
```


::: {.panel-tabset}

## Plot


```{ojs}
f = (x) => Number.isInteger(x) ? x.toPrecision(2) : x

Plot.plot({
  x: {type:"log"},
  y: {domain: [0,1]},
  color: {scheme: "reds", type: "ordinal", legend: true},
  marks: [
    Plot.line(
      data[`${f(mu)}_${f(nu_n)}_${f(nu_p)}_${f(alpha)}_${f(beta)}`], {
        x: 'timesteps', y: "value", stroke: "N"
        }),
    Plot.dot(
      data[`${f(beta)}_${f(gamma)}_${f(rho)}_${f(b)}_${f(c)}_${f(mu)}`], {
        x: 'timesteps', y: "value", stroke: "N"
        })
  ]
})
```



:::

##  Takeaways:
